# Create classes

In [1]:
import json
import numpy as np
import random

In [2]:
from IPython.core.debugger import set_trace

In [3]:
%config IPCompleter.greedy=True

In [ ]:
# %load mesh.py

class Mesh:
    
    def __init__(self, *args, **kwargs):
        """
        Constructor for Mesh class
        """
        self.spokes  = 2
        self.nodes   = 10
        self.d_nnode = {}
        self.l_nodes = []
        
        self.d_mesh  = {}
        
        for k,v in kwargs.items():
            if k == 'spokes': self.spokes = v
            if k == 'nodes':  self.nodes  = v
        
        self.l_nodes = list(range(self.nodes))
        self.d_nnode = dict(enumerate(self.l_nodes))
        
        print(self.l_nodes)
        print(self.d_nnode)        
        
    def connect(self, *args, **kwargs):
        """
        Connect each node to a random sample (of size <spoke>)
        drawn from list of all nodes.
        """
        for k,v in self.d_nnode.items():
            self.d_nnode[k] = random.sample(self.l_nodes, self.spokes)
            
    def FDGnodelist_build(self, *args, **kwargs):
        """
        Build the list of nodes for the FDG.
        
        Group ID can be 'uniform', with all values set to <groupID>,
        or this can be 'linear', starting with <groupID> and
        increasing with <incremenet>. Group ID strings can also
        be prepended with an optional <prefix>.
        """
        str_groupSpread = 'uniform'
        str_prefix      = ''
        groupID         = 1
        increment       = 1
        for k,v in kwargs.items():
            if k == 'groupSpread': str_groupSpread = v
            if k == 'groupID':     groupID         = v
            if k == 'prefix':      str_prefix      = v
            if k == 'increment':   increment       = v

        lstr_ID = [str_prefix] * self.nodes
        if str_groupSpread == 'uniform':
            l_ID = [groupID] * self.nodes
        if str_groupSpread == 'increment':
            l_ID = list(range(groupID, groupID + self.nodes*increment, increment))
            
        t_ID     = zip(lstr_ID, l_ID)
        l_fullID = ['%s%d' % (s, i) for (s, i) in zip(lstr_ID, l_ID)]
        set_trace()
        return l_fullID           
            

In [5]:
M = Mesh(spokes = 2, nodes = 10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}


In [6]:
M.connect()

In [7]:
M.d_nnode

{0: [2, 9],
 1: [1, 0],
 2: [6, 7],
 3: [6, 5],
 4: [1, 5],
 5: [9, 8],
 6: [4, 2],
 7: [6, 4],
 8: [8, 6],
 9: [1, 7]}

In [11]:
M.FDGnodelist_build(prefix='group', groupSpread = 'increment', increment=2)

> <ipython-input-4-bbc336da089e>(60)FDGnodelist_build()
     57         t_ID     = zip(lstr_ID, l_ID)
     58         l_fullID = ['%s-%d' % (s, i) for (s, i) in zip(lstr_ID, l_ID)]
     59         set_trace()
---> 60         return l_fullID
     61 

ipdb> c


['group-1',
 'group-3',
 'group-5',
 'group-7',
 'group-9',
 'group-11',
 'group-13',
 'group-15',
 'group-17',
 'group-19']

In [8]:
['tt'] * 4

['tt', 'tt', 'tt', 'tt']

In [115]:
l_nodesCloudHeterogeneous = []
l_linksCloudHeterogeneous = []
l_nodesCloudHomogeneous   = []
l_linksCloudHomogeneous   = []
cloudNodes                = 300
edgeNodes                 = 30
linkToEdgeNodes           = 10
d_graphDisconnected       = {}
d_graphCentralServer      = {}
d_graphCloudHomogeneous   = {}
d_graphEdge               = {}
d_graphFog                = {}

In [116]:
for i in range(1, cloudNodes):
    l_nodesCloudHeterogeneous.append({"id": "pc%d" % i, "group": i})
    l_nodesCloudHeterogeneous.append({"id": "human%d" % i, "group": i})
    l_linksCloudHeterogeneous.append({"source": "pc%d" % i, "target": "human%d" % i, "value": 10})

In [117]:
d_graphDisconnected = {
    "nodes": l_nodesCloudHeterogeneous,
    "links": l_linksCloudHeterogeneous
}
with open('pcss.json', 'w') as f:
    json.dump(d_graphDisconnected, f, sort_keys=True, indent=4)

# Create a central server

In [118]:
l_nodesCloudHeterogeneous.append({"id": "server1", "group": 200})

In [119]:
for i in range(1, cloudNodes):
    l_linksCloudHeterogeneous.append({"source": "pc%d" % i, "target": "server1", "value": 1})

In [120]:
d_graphCentralServer = {
    "nodes": l_nodesCloudHeterogeneous,
    "links": l_linksCloudHeterogeneous
}
with open('pcss-net.json', 'w') as f:
    json.dump(d_graphCentralServer, f, sort_keys=True, indent=4)

# Create a "cloud" topology

# First create the node and links out from the cloud to the edge

In [121]:
# Create a homogeneous cloud
l_nodesCloudHomogeneous.append({"id": "headnode", "group": 100})
for i in range(1, cloudNodes):
    l_nodesCloudHomogeneous.append({"id": "node%d" %i, "group": 1})
    l_linksCloudHomogeneous.append({"source": "node%d" % i, "target": "headnode", "value": 1})

In [122]:
# Create the nodes out of the cloud
for i in range(1, linkToEdgeNodes):
    l_nodesCloudHomogeneous.append({"id": "link%d" % i, "group": cloudNodes})
l_nodesCloudHomogeneous.append({"id": "client", "group": 400})

In [123]:
# Link the nodes to create a chain out of the cloud
for i in range(1, linkToEdgeNodes-1):
    l_linksCloudHomogeneous.append({"source": "link%d" % i, "target": "link%s" % str(i+1)})
l_linksCloudHomogeneous.append({"source": "link1", "target": "headnode", "value": 1})
l_linksCloudHomogeneous.append({"source": "link%s" % str(linkToEdgeNodes-1), "target": "client", "value": 1})

In [124]:
d_graphCloudHomogeneous = {
    "nodes": l_nodesCloudHomogeneous,
    "links": l_linksCloudHomogeneous
}
with open('cloud.json', 'w') as f:
    json.dump(d_graphCloudHomogeneous, f, sort_keys=True, indent=4)

# Create another cloud for the "edge" computing

In [125]:
for i in range(1, edgeNodes):
    l_nodesCloudHomogeneous.append({"id": "edgeNode%d" % i, "group": 2})
    l_linksCloudHomogeneous.append({"source": "edgeNode%d" %i, "target": "link%s" % str(linkToEdgeNodes-1), "value": 1})

In [126]:
d_graphEdge = {
    "nodes": l_nodesCloudHomogeneous,
    "links": l_linksCloudHomogeneous
}
with open('edge.json', 'w') as f:
    json.dump(d_graphEdge, f, sort_keys=True, indent=4)

# Fog computing

In [130]:
for i in range(1, edgeNodes):
    l_nodesCloudHomogeneous.append({"id": "fogNode3.%d" %i, "group": 3})
    l_linksCloudHomogeneous.append({"source": "fogNode3.%d" %i, "target": "link3", "value": 1})
    l_nodesCloudHomogeneous.append({"id": "fogNode6.%d" %i, "group": 4})
    l_linksCloudHomogeneous.append({"source": "fogNode6.%d" %i, "target": "link6", "value": 1})

In [131]:
d_graphFog = {
    "nodes": l_nodesCloudHomogeneous,
    "links": l_linksCloudHomogeneous
}
with open('fog.json', 'w') as f:
    json.dump(d_graphFog, f, sort_keys=True, indent=4)